## Example SORTS visualizer

### Based on the "Live interaction demo of Space Object Radar Tracking Simulator (SORTS)" presentation by: D. Kastinen, J. Vierinen, T. Grydeland, J. Kero. Code by: Daniel Kastinen


In [1]:
#Setup widgets
from __future__ import print_function
import ipywidgets as widgets
from IPython.display import display

#setup basic plotting
%matplotlib widget
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d

import numpy as np

#Load SORTS
import sorts

print(f'Using SORTS: {sorts.__version__}')

Using SORTS: 4.0.0


### SORTS

__A toolbox for simulating radars, their schedule, objects in space and resulting measurements from those radars__

* The idea behind this presentation: show how scheduling can be simulated with SORTS
* Took me about <span style="color:#ff2222">1h</span> to code the simulation using SORTS

[SORTS on github](https://github.com/danielk333/SORTS)
* To get the code / contribute / suggest

[SORTS examples (and docs)](https://danielk333.github.io/SORTS/docs/auto_gallery/index.html)
* For your inspiration

[SORTS in peer-review!](https://angeo.copernicus.org/articles/38/861/2020/)
* Cool application!

[SORTS and ESA projects](https://conference.sdo.esoc.esa.int/proceedings/neosst1/paper/465)
* 10,000 - 20,000 debris passes per hour over E3D


In [2]:
#Set the path to our population, alternativly define a custom one
path = '/home/danielk/IRF/IRF_GITLAB/SORTSpp/master/celn_20090501_00.sim'

from scheduler_simulation import get_master_population

pop = get_master_population(path)

In [3]:
#The population class has a print function, lets look at the docs
help(pop.print)

Help on method print in module sorts.population.population:

print(n=None, fields=None) method of sorts.population.population.Population instance



In [4]:
#look at a few objects from the population
print(pop.print(list(range(10)), fields=['oid', 'a', 'e', 'i', 'd']))

  oid            a       e      i      d
-----  -----------  ------  -----  -----
    0  4.05806e+07  0.0043   3.68  33.57
    1  3.95586e+07  0.0579   9.28  31.06
    2  7.1087e+06   0.0043  70.94  27.64
    3  2.79227e+07  0.4739  70.8   27.52
    4  7.0614e+06   0.0031  70.94  27.51
    5  7.3753e+06   0.0225  98.73  27.45
    6  4.3401e+07   0.0215  14.51  26.42
    7  4.22708e+07  0.0282  13.6   24.43
    8  1.33294e+07  0.4859  51.58  24.4
    9  1.58685e+07  0.5696  64.88  23.84


In [5]:
#Set some simulation parameters
dt = 20.0 #time step [seconds] for orbit sampling
END_T = 5*60.0 #end time in minutes

In [6]:
#Import our simulation into the notebook
from scheduler_simulation import calc_observation, get_scheduler, recalc_track_data, get_widget_limits
eiscat3d = sorts.radars.eiscat3d

e3d_sched = get_scheduler(eiscat3d, pop, dt, END_T)

In [24]:
#Setup all the matplotlib stuff
output = widgets.Output()
with output:
    fig = plt.figure(figsize=(8,6))
    ax = fig.add_subplot(111, projection='3d')
    ax2 = fig.add_axes([.7, .1, .2, .2])

def fun(**kw):
    calc_observation(e3d_sched, ax, ax2, **kw)

grid = widgets.GridspecLayout(8,2)

#grid[8,:] = output

app = widgets.AppLayout(
    header=None,
    left_sidebar=None,
    center=output,
    right_sidebar=grid,
    footer=None,
    height="600px", 
    width="100%",
)

#get the prepared widgets limits
lims = get_widget_limits(pop, END_T)

###
# CREATE ALL THE SLIDERS AND UI.... AAAAAARGH!
###
t = widgets.FloatSlider(min=lims['t'][0], max=lims['t'][1], step=lims['t'][2], description='Time [s]'); grid[1,-2] = t
t_cnt = widgets.FloatText(min=lims['t'][0], max=lims['t'][1], step=lims['t'][2], description='Time [s]'); grid[0,-1] = t_cnt
t_span = widgets.FloatSlider(min=lims['t_span'][0], max=lims['t_span'][1], step=lims['t_span'][2], description='Time span [s]'); grid[2,-2] = t_span
view_range = widgets.FloatSlider(min=lims['view_range'][0], max=lims['view_range'][1], step=lims['view_range'][2], description='Zoom [km]'); grid[1,-1] = view_range
d_min = widgets.FloatSlider(min=lims['d_min'][0], max=lims['d_min'][1], step=lims['d_min'][2], value=5.0, description='Min diameter [m]'); grid[2,-1] = d_min
L = widgets.FloatSlider(min=lims['L'][0], max=lims['L'][1], step=lims['L'][2], description='Side-length [km]'); grid[3,-2] = L
N = widgets.IntSlider(min=lims['N'][0], max=lims['N'][1], step=lims['N'][2], description='Samples/side [1]'); grid[3,-1] = N
dwell = widgets.FloatSlider(min=lims['dwell'][0], max=lims['dwell'][1], step=lims['dwell'][2], description='Dwell [s]'); grid[4,-2] = dwell
h = widgets.FloatSlider(min=lims['h'][0], max=lims['h'][1], step=lims['h'][2], description='Altitude [km]'); grid[4,-1] = h
h_min = widgets.FloatSlider(min=lims['h_min'][0], max=lims['h_min'][1], step=lims['h_min'][2], description='Min altitude [km]'); grid[5,-2] = h_min
beams = widgets.IntSlider(min=lims['beams'][0], max=lims['beams'][1], step=lims['beams'][2], description='Rx-beams [1]'); grid[5,-1] = beams
plot_rx = widgets.ToggleButton(value=True, description='Plot rx beams'); grid[6,-2] = plot_rx
include_objects = widgets.ToggleButton(value=False, description='Plot objects'); grid[6,-1] = include_objects
sim_snr = widgets.ToggleButton(value=False, description='Simulate observation'); grid[7,-2] = sim_snr
recalc = widgets.Button(description='Run Observation simulation', button_style=''); grid[7,-1] = recalc
track = widgets.Dropdown(
    options=['None'] + e3d_sched.inds,
    value = 1268,
    description='Track object:',
)
grid[0,-2:] = track


#Relations
def h_on_value_change(change):
    new_h = change['new']
    h_min.max = new_h
h.observe(h_on_value_change, names='value')

def on_button_clicked(b):
    recalc_track_data()
recalc.on_click(on_button_clicked)

tlink = widgets.jslink((t, 'value'), (t_cnt, 'value'))

#Cache interactive to start with
out = widgets.interactive_output(
    fun, 
    dict(
        t = t,
        t_span = t_span,
        d_min = d_min,
        L = L,
        N = N,
        dwell = dwell,
        h = h,
        h_min = h_min,
        beams = beams,
        plot_rx = plot_rx,
        include_objects = include_objects,
        view_range = view_range,
        track_ind = track,
        track_snr = sim_snr,
    ),
)

#Display control
display(app, out)

AppLayout(children=(GridspecLayout(children=(FloatSlider(value=0.0, description='Time [s]', layout=Layout(grid…

Output()


# Discussion

## Questions

* Thoughts about scheduling for your own future experiments? <span style="color:#ff2222">Lets discuss!</span>
* Does this framework seem useful to your research? how? <span style="color:#ff2222">Poke me!</span>
* Could it be useful with modification/additions? what modifications? <span style="color:#ff2222">Open an issue on Github!</span>